<a href="https://colab.research.google.com/github/talhaanwarch/ODIR2019/blob/master/duplicate_minority_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#connect drive with colab notebook/

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


https://github.com/keras-team/keras/issues/8130
https://stackoverflow.com/questions/56796331/how-to-input-multiple-images-with-flow-from-dataframe-in-keras

In [2]:
#following are libraries to unpack 7z file
!pip install patool 
!pip install pyunpack
from pyunpack import Archive

In [0]:
import pandas as pd
import numpy as np
import cv2

In [4]:

#create a folder name as data
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [0]:
#path where our data is saved
path='/content/drive/My Drive/ghina/thesis/'

In [0]:
#extract the 7z file in data folder we created above
#this not only extract the data but move it from drive to colab for efficinet processing
#it will take some time
Archive(path+'ODIR-5K_Training_Images.7z').extractall('data')

In [0]:
#read the annottion file
annotation=pd.read_excel(path+'ODIR-5K_Training_Annotations(Updated)_V2.xlsx')

In [0]:

#normal (N), diabetes (D), glaucoma (G), cataract (C), AMD (A), hypertension (H), myopia (M) and other diseases/abnormalities (O)

In [27]:
#print first 5 rows of annotation file
annotation=annotation.sample(frac=1)
annotation.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
684,686,37,Male,686_left.jpg,686_right.jpg,hypertensive retinopathy,hypertensive retinopathy,0,0,0,0,0,1,0,0
1516,1919,50,Male,1919_left.jpg,1919_right.jpg,refractive media opacity,dry age-related macular degeneration,0,0,0,0,1,0,0,1
1697,2242,63,Male,2242_left.jpg,2242_right.jpg,cataract,cataract,0,0,0,1,0,0,0,0
1824,2447,34,Female,2447_left.jpg,2447_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
125,125,71,Male,125_left.jpg,125_right.jpg,epiretinal membrane,normal fundus,0,0,0,0,0,0,0,1


In [0]:
train_annotation=annotation[0:2500]
val_annotation=annotation[2500:3500]


In [47]:
train_annotation.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
684,686,37,Male,686_left.jpg,686_right.jpg,hypertensive retinopathy,hypertensive retinopathy,0,0,0,0,0,1,0,0
1516,1919,50,Male,1919_left.jpg,1919_right.jpg,refractive media opacity,dry age-related macular degeneration,0,0,0,0,1,0,0,1
1697,2242,63,Male,2242_left.jpg,2242_right.jpg,cataract,cataract,0,0,0,1,0,0,0,0
1824,2447,34,Female,2447_left.jpg,2447_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
125,125,71,Male,125_left.jpg,125_right.jpg,epiretinal membrane,normal fundus,0,0,0,0,0,0,0,1


In [0]:
for i in ['G',	'C',	'A',	'H',	'M']:
  duplicate=train_annotation[train_annotation[i]==1]
  train_annotation=train_annotation.append([duplicate]*2,ignore_index=True)

In [0]:

# train_annotation.drop(columns=['ID', 'Patient Age', 'Patient Sex','Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'],inplace=True)
# val_annotation.drop(columns=['ID', 'Patient Age', 'Patient Sex','Left-Diagnostic Keywords', 'Right-Diagnostic Keywords'],inplace=True)

# def change_name(x):
#   l=list(x)
#   l.insert(l.index('_'),'_1')
#   l=''.join(l)
#   return l
# def df_row_duplicate(df):
#   for i in range(len(df)):
#     left=change_name(df.iloc[i,0]) #creat new image name

#     img_left=Image.open('/content/data/ODIR-5K_Training_Dataset/{}'.format(df.iloc[i,0]))
#     img_left = img_left.save('/content/data/ODIR-5K_Training_Dataset/{}'.format(img_left)) 

#     right=change_name(df.iloc[i,1])
#     img_right=Image.open('/content/data/ODIR-5K_Training_Dataset/{}'.format(df.iloc[i,1]))
#     img_right = img_right.save('/content/data/ODIR-5K_Training_Dataset/{}'.format(img_right)) 

#     label=list(df.iloc[i,-8::])
#     new_list=[left,right]+label
#     df=df.append(pd.Series(new_list,index=['Left-Fundus',	'Right-Fundus','N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']),ignore_index=True)
#   return df

In [49]:
len(train_annotation)

3820

In [0]:
from keras_preprocessing.image import ImageDataGenerator

train_data_gen= ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    rescale=1/255,
    fill_mode='nearest'
)

val_data_gen= ImageDataGenerator(
    rescale=1/255
    
)

In [0]:

def get_flow_from_dataframe(generator, dataframe,directory_path,
                            image_shape=(224, 224),
                            #subset='training',
                             batch_size=50):
    train_generator_1 = generator.flow_from_dataframe(dataframe,
                                                      directory=directory_path,
                                                      target_size=image_shape,
                                                      x_col="Left-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )

    train_generator_2 = generator.flow_from_dataframe(dataframe, 
                                                      directory=directory_path,
                                                      target_size=image_shape,
                                                      x_col="Right-Fundus",
                                                      y_col=['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O'],
                                                      class_mode='other',
                                                      shuffle=False,
                                                      batch_size=batch_size,
                                                      seed=7,
                                                      #subset=subset
                                                      )
    while True:
        x_1 = train_generator_1.next()
        x_2 = train_generator_2.next()

        yield [x_1[0], x_2[0]], x_1[1]

In [0]:
batch_size=50
train_gen = get_flow_from_dataframe(train_data_gen, train_annotation, image_shape=(224, 224),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)
valid_gen = get_flow_from_dataframe(val_data_gen, val_annotation, image_shape=(224, 224),
                                        directory_path="data/ODIR-5K_Training_Dataset/",batch_size=batch_size)

In [0]:
from keras.models import Sequential,Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization,Input,MaxPooling2D,GlobalMaxPooling2D,concatenate
from keras.layers import GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [0]:
inp1= Input(shape=(224, 224, 3))
inp2= Input(shape=(224, 224, 3))

X1=BatchNormalization()(inp1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=MaxPooling2D(pool_size=(2, 2))(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=Conv2D(32, (3, 3), padding='same',activation='relu')(X1)
X1=GlobalMaxPooling2D()(X1)


X2=BatchNormalization()(inp2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=MaxPooling2D(pool_size=(2, 2))(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=Conv2D(32, (3, 3), padding='same',activation='relu')(X2)
X2=GlobalMaxPooling2D()(X2)

X=concatenate([X1,X2])
X=Dropout(0.25)(X)
X=Dense(512,activation='relu')(X)
X=Dropout(0.25)(X)
out=Dense(8, activation='sigmoid')(X)

model=Model(inputs=[inp1,inp2],outputs=out)



In [0]:
import keras.backend as K
def focal_loss(y_true, y_pred, alpha = 0.75, gamma = 2):
  alpha_factor = K.ones_like(y_true) * alpha
  alpha_factor = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
  focal_weight = tf.where(K.equal(y_true, 1), 1 - y_pred, y_pred)
  focal_weight = alpha_factor * focal_weight ** gamma
  loss = focal_weight * K.binary_crossentropy(y_true, y_pred)
  return loss

In [0]:
model.compile('Adadelta',loss=focal_loss,metrics=['accuracy'])

In [0]:
step_size_train=len(train_annotation)/batch_size
step_size_val=1000/50


In [0]:
model.fit_generator(train_gen, epochs=5,
                        steps_per_epoch=step_size_train,
                        use_multiprocessing=True,
                        validation_data=valid_gen,
                        validation_steps=step_size_val,
                        verbose=2,
                    class_weight=[0.5,0.5,1.5,1.5,1.5,2.5,1.5,0.5]
                    
                        )

Epoch 1/5
Found 1000 validated image filenames.
Found 1000 validated image filenames.
Found 3820 validated image filenames.
Found 3820 validated image filenames.
 - 580s - loss: 0.0507 - acc: 0.2744 - val_loss: 0.0618 - val_acc: 0.0440
Epoch 2/5


In [20]:
y_pred = model.predict_generator(valid_gen,steps=step_size_val)


Found 1000 validated image filenames.
Found 1000 validated image filenames.


In [0]:
y_pred1=y_pred.copy()
y_pred1[y_pred1>=0.35] = 1
y_pred1[y_pred1<0.35] = 0
y_pred1=y_pred1.astype(int)

In [22]:
y_true=val_annotation.iloc[:,-8::].values
y_true.shape

(1000, 8)

In [23]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_true, y_pred1)

array([[[  0, 668],
        [  0, 332]],

       [[  0, 695],
        [  0, 305]],

       [[425, 516],
        [ 15,  44]],

       [[530, 404],
        [ 19,  47]],

       [[695, 257],
        [ 38,  10]],

       [[963,  12],
        [ 25,   0]],

       [[793, 161],
        [ 38,   8]],

       [[  0, 711],
        [  0, 289]]])

In [24]:
target_names = ['N'	,'D'	,'G',	'C',	'A'	,'H'	,'M',	'O']
print(classification_report(y_true, y_pred1, target_names=target_names))

              precision    recall  f1-score   support

           N       0.33      1.00      0.50       332
           D       0.30      1.00      0.47       305
           G       0.08      0.75      0.14        59
           C       0.10      0.71      0.18        66
           A       0.04      0.21      0.06        48
           H       0.00      0.00      0.00        25
           M       0.05      0.17      0.07        46
           O       0.29      1.00      0.45       289

   micro avg       0.23      0.88      0.37      1170
   macro avg       0.15      0.61      0.23      1170
weighted avg       0.26      0.88      0.40      1170
 samples avg       0.25      0.89      0.38      1170



In [25]:
(0.49+0.47+.12+.06+.00+.10+0.47)/8

0.21375000000000002

In [0]:
K.clear_session()
